# Configurações

## Importações

In [1]:
import os
from dotenv import load_dotenv
import time

import requests
import json

import pandas as pd

## Carregamento de variáveis de ambiente

In [2]:
API_KEY = os.getenv("TMDB_API_KEY")
BASE_URL = os.getenv("BASE_URL")

In [3]:
print("--- Coletando, Processando Dados de Filmes e Aplicando ML com a API TMDb ---")
print("-------------------------------------------------------------------------\n")

print(f"Usando BASE_URL: {BASE_URL}")
print(f"Usando API_KEY: {API_KEY}")

--- Coletando, Processando Dados de Filmes e Aplicando ML com a API TMDb ---
-------------------------------------------------------------------------

Usando BASE_URL: https://api.themoviedb.org/3
Usando API_KEY: c660b2a02a8f4e1313151be1ea06776f


## Coleta de dados

In [4]:
# --- Parte 1: Coleta de Dados (Reutilizando código anterior) ---
print("Coleta de Dados de Filmes Populares do TMDb\n")

popular_movies_endpoint = f"{BASE_URL}/movie/popular"
all_movies_data = []
num_pages_to_fetch = 5  # Aumentamos para ter mais dados para ML

print(f"Coletando dados das primeiras {num_pages_to_fetch} páginas de filmes populares...")

for page_num in range(1, num_pages_to_fetch + 1):
    params = {
        'api_key': API_KEY,
        'language': 'pt-BR',
        'page': page_num
    }
    print(f"   Buscando página {page_num}...")
    
    try:
        response = requests.get(popular_movies_endpoint, params=params)
        response.raise_for_status()
        page_data = response.json()
        all_movies_data.extend(page_data['results'])
        print(f"   Página {page_num} coletada. {len(page_data['results'])} filmes adicionados.")
        time.sleep(0.5)  # Pausa de 0.5 segundos
        
    except requests.exceptions.HTTPError as e:
        print(f"   ERRO HTTP ao buscar página {page_num}: {e.response.status_code} - {e.response.text}")
        print("   Parando a coleta de páginas.")
        break
        
    except requests.exceptions.RequestException as e:
        print(f"   ERRO de Requisição ao buscar página {page_num}: {e}")
        print("   Parando a coleta de páginas.")
        break

print(f"\nColeta finalizada. Total de {len(all_movies_data)} filmes coletados.")

Coleta de Dados de Filmes Populares do TMDb

Coletando dados das primeiras 5 páginas de filmes populares...
   Buscando página 1...
   Página 1 coletada. 20 filmes adicionados.
   Buscando página 2...
   Página 2 coletada. 20 filmes adicionados.
   Buscando página 3...
   Página 3 coletada. 20 filmes adicionados.
   Buscando página 4...
   Página 4 coletada. 20 filmes adicionados.
   Buscando página 5...
   Página 5 coletada. 20 filmes adicionados.

Coleta finalizada. Total de 100 filmes coletados.


# Definição do Dataframe

In [5]:
# 3. Criar um DataFrame Pandas com os dados coletados
selected_columns_raw = [
    'id', 'title', 'overview', 'release_date', 'vote_average',
    'vote_count', 'popularity', 'original_language', 'genre_ids',
    'adult'  # Adicionamos 'adult' para um possível uso em ML
]

processed_movies_data = []
for movie in all_movies_data:
    filtered_movie = {col: movie.get(col) for col in selected_columns_raw}
    processed_movies_data.append(filtered_movie)

df_movies = pd.DataFrame(processed_movies_data)

print("\n3. DataFrame Pandas inicial criado com sucesso:")


3. DataFrame Pandas inicial criado com sucesso:


In [6]:
df_movies.head()

,id,title,overview,release_date,vote_average,vote_count,popularity,original_language,genre_ids,adult
0,1156594,Nossa Culpa,O casamento de Jenna e Lion marca o tão espera...,2025-10-15,7.900,306,1096.6654,es,"[10749, 18]",False
1,1072699,Furioza 2,"Após um assassinato, o novo líder da Furioza, ...",2025-10-14,6.593,27,384.3220,pl,"[28, 80, 53]",False
2,1511789,Captain Hook - The Cursed Tides,Após uma derrota devastadora sofrida por seu a...,2025-07-11,5.000,15,370.1091,en,"[12, 28, 27]",False
3,755898,A Guerra dos Mundos,Will Radford é um renomado analista de ciberse...,2025-07-29,4.353,645,327.8760,en,"[878, 53]",False
4,1038392,Invocação do Mal - O Último Ritual,"Neste último capítulo, os Warren enfrentam mai...",2025-09-03,6.975,1069,284.0186,en,[27],False


In [7]:
print(f"Formato do DataFrame: {df_movies.shape}")

Formato do DataFrame: (100, 10)


# Gerar o dataset

In [8]:
os.makedirs('../datasets', exist_ok=True)
df_movies.to_csv("../datasets/movies.csv")